# Make sure the following packages are installed before trying to run this code:
___

* from neuralprophet import NeuralProphet
* from pmutils import redshift
* from pandas.plotting import register_matplotlib_converters
* import pandas as pd
* import numpy as np
* from math import floor
* from scipy.optimize import curve_fit
* from neuralprophet import set_random_seed 
* set_random_seed(0)

In [2]:
def metric_forecast(input_df, date_field, val_field, freq, pred_time, model_metrics = False):
    ts_df = input_df[[date_field, val_field]]
    ts_df.columns = ['ds', 'y']
    
    m = NeuralProphet(
        daily_seasonality = False,
        batch_size = 32,
        epochs = 55
    )
    
    metrics = m.fit(
        ts_df,
        freq = freq,
        validate_each_epoch = True,
        valid_p = 0.2
    )
    
    future = m.make_future_dataframe(
        ts_df, 
        periods=pred_time, 
        n_historic_predictions=len(ts_df)
    )
    
    forecast = m.predict(future)
    
    if model_metrics:
        return metrics, forecast
    else:
        return forecast

In [3]:
def ts_pre_post_impact(input_df, date_field, val_field, freq, pred_time, start_date, end_date, pre_time = False):
    output_df = metric_forecast(input_df, date_field, val_field, freq, pred_time, model_metrics = False)
    
    post_period = output_df[
        (pd.to_datetime(output_df['ds']) >= start_date) &
        (pd.to_datetime(output_df['ds']) < end_date)
    ]
    
    pre_period = output_df[pd.to_datetime(output_df['ds']) < start_date]
    pre_mean = pre_period['residual1'].mean()
    pre_std = pre_period['residual1'].std()
    
    pre_period['residual_std'] = (pre_period['residual1'] - pre_mean)/pre_std
    
    post_period['residual_std'] = (post_period['residual1'] - pre_mean)/pre_std
    
    if pre_time:
        return pre_period, post_period
    else:
        return post_period

In [4]:
def window_impact(input_df, date_field, val_field, freq, pred_time, start_date, end_date):
    pre_input, post_input = ts_pre_post_impact(
        input_df, 
        date_field, 
        val_field, 
        freq, 
        pred_time, 
        start_date, 
        end_date, 
        pre_time = True
    )
    
    pre_residual_mean = pre_input['residual1'].mean()
    post_residual_mean = post_input['residual1'].mean()
    pre_post_impact = post_input['residual_std'].mean() - pre_input['residual_std'].mean()
    
    return pre_post_impact, pre_residual_mean, post_residual_mean

In [5]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

In [6]:
def compare_df_performance(list_of_dfs, date_field, val_field, freq, pred_time, start_date, end_date):
    
    col_names = ["dataset", "prepost_impact", "pre_difference", "post_difference", "start_date", "end_date"]
    result_df = pd.DataFrame(columns = col_names)

    for i in list_of_dfs:
        impact_val, pre_impact, post_impact = window_impact(
            input_df = i, 
            date_field = 'event_date', 
            val_field = 'gmv', 
            freq = 'D', 
            pred_time = 28, 
            start_date = start_date, 
            end_date = end_date
        )

        result_df = result_df.append(
            {
                'dataset': get_df_name(i),
                'prepost_impact': impact_val,
                'pre_difference': pre_impact,
                'post_difference': post_impact,
                'start_date': start_date,
                'end_date': end_date
            },
            ignore_index = True
        )

    return result_df

# Code Template:
___

Create a list of the dataframes you will be checking:

df_list = [
    gmv_df,
    org_gmv_df,
    paid_gmv_df,
    org_web_gmv_df,
    org_ios_gmv_df,
    org_and_gmv_df,
    paid_web_gmv_df,
    paid_ios_gmv_df,
    paid_and_gmv_df
]

# Code For Running the Function:
___

```
import warnings
warnings.filterwarnings('ignore')

test_output = compare_df_performance(
    list_of_dfs = df_list, 
    date_field = 'event_date', 
    val_field = 'gmv', 
    freq = 'D', 
    pred_time = 7, 
    start_date = '2020-11-27', 
    end_date = '2020-12-13'
)

warnings.filterwarnings('default')
```